In [ ]:
import os
from datetime import datetime
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
#from osgeo import gdal
import rasterio as rio
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
import pyproj
from pyproj import Proj
from shapely.ops import transform
import xarray as xr
#import geoviews as gv
#from cartopy import crs
import hvplot.xarray
import holoviews as hv
import boto3
from rasterio.session import AWSSession

# Section 1: Navigating the CMR-STAC API

## Set GDAL Configuration Options

In [ ]:
r.packages.urllib3.disable_warnings()

def get_temp_creds():
    temp_creds_url = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'
    return r.get(temp_creds_url, verify=False).json()

temp_creds_req = get_temp_creds()
#temp_creds_req

session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

rio_env = rio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  CPL_VSIL_CURL_ALLOWED_EXTENSIONS='tif',
                  GDAL_HTTP_UNSAFESSL='YES',
                  VSI_CACHE=True,
                  region_name='us-west-2',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

## 1. STAC API: Endpoint that enables the querying of STAC items

In [ ]:
stac = 'https://cmr.earthdata.nasa.gov/stac/' # CMR-STAC API Endpoint
r.packages.urllib3.disable_warnings()  # Disable SSL warnings
stac_response = r.get(stac, verify=False).json() # Call the STAC API endpoint
for s in stac_response: print(s)

print(f"You are now using the {stac_response['id']} API (STAC Version: {stac_response['stac_version']}). {stac_response['description']}")
print(f"There are {len(stac_response['links'])} STAC catalogs available in CMR.")

## 2. STAC Catalog: Contains a JSON file of links that organize all of the collections available

In [ ]:
stac_lp = [s for s in stac_response['links'] if 'LP' in s['title']]  # Search for only LP-specific catalogs

# LPCLOUD is the STAC catalog we will be using and exploring today
lp_cloud = r.get([s for s in stac_lp if s['title'] == 'LPCLOUD'][0]['href'], verify=False).json()
for l in lp_cloud: print(f"{l}: {lp_cloud[l]}")

# Print the links contained in the LP CLOUD STAC Catalog
lp_links = lp_cloud['links']
for l in lp_links: 
    try: 
        print(f"{l['href']} is the {l['title']}")
    except:
        print(f"{l['href']}")

## 3. STAC Collection: Extension of STAC Catalog containing additional information that describe the STAC items in that collection

In [ ]:
# Get a response form the LPCLOUD collection and print the information included in the response
lp_collections = [l['href'] for l in lp_links if l['rel'] == 'collections'][0]  # Set collections endpoint to variable
print(lp_collections)
collections_response = r.get("https://cmr.earthdata.nasa.gov/stac/LPCLOUD/collections/?page=2", verify=False).json()                        # Call collections endpoint
print(collections_response)
print(f"This collection contains {collections_response['description']} ({len(collections_response['collections'])} available)")

collections = collections_response['collections']
print("\n")

# Search available version 2 collections for HLS and print them out
hls_collections = [c for c in collections if 'HLS' in c['id'] and 'v2' in c['id']]
#print(hls_collections)
for h in hls_collections: 
    print(f"{h['title']} has an ID (shortname) of: {h['id']}")

# Explore the attributes contained in the HLSS30 collection
s30 = [h for h in hls_collections if 'HLSS30' in h['id']][0]    # Grab HLSS30 collection

for s in s30['extent']:                                         # Check out the extent of this collection
    print(f"{s}: {s30['extent'][s]}")

print(f"HLS S30 Start Date is: {s30['extent']['temporal']['interval'][0][0]}")

l30 = [h for h in hls_collections if 'HLSL30' in h['id'] and 'v2.0' in h['id']][0]     # Grab HLSL30 collection

for l in l30['extent']:                                                                # Check out the extent of this collection
    print(f"{l}: {l30['extent'][l]}")

## 4. STAC ITem: Represents data and metadata assets that are spatiotempporally coincident